# Amazon Uk의 신발 제품 리뷰 데이터 감정 분석(Sentiment Analysis)

### [건국대학교 글로컬캠퍼스 컴퓨터공학과 202121859 이을호]

========================================================================

본 과제에서는 Amazon uk(영국)쇼핑몰의 신발 제품 판매 리뷰에 대한 긍정, 부정의 판별이 가능한 감정 분석 시스템 개발을 진행한다. 

모델의 구현은 하단의 과정을 순서로 개발을 진행한다.

1. 데이터 전처리 (결측치 처리, 노이즈 제거, 정규화 과정)

```데이터 전처리 과정에서 신발 제품 별 리뷰 갯수, 실구매 여부, 동일 작성자 리뷰 갯수 확인을 진행함.```

2. 모델 구현 (불용어 제거 진행)

3. 모델 학습 및 튜닝 (dataset을 train, test로 분할, 모델 학습 및 하이퍼파라미터 튜닝(grid search or random search)를 통한 교차 검증)

4. 모델 평가 (정확도, 정밀도, 재현율, F1 score, ROC-AUC 지표를 통한 모델 검증 및 overfitting, underfitting 여부 확인 및 해결 방법)

5. 시각화 및 보고 (모델이 출력한 결과 및 수치값을 matplotlib, Seaborn을 통한 시각화 진행(혼동 행렬 및 ROC 곡선 생성))

-> 상기 내용을 보고서로 작성하여 별도 첨부함.

========================================================================

- Python Version : Python 3.10.13

# 1. 데이터 전처리

In [1]:
# 필요 라이브러리 import

%pip install matplotlib
%pip install pandas

import pandas as pd # pandas 모듈 import
import time # (작업 소요시간 계산) time 모듈 import
import ast # 구문 트리 파싱 및 분석에 사용하는 asr 모듈 import

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### 1) 데이터 구조 확인

데이터를 우선 살펴보면 "Amazon_uk_shoes_products_dataset_2021_12.csv"는 하단의 순서대로 데이터가 이루어져 있음을 확인 가능하다.

1. url (상품 URL 주소)
2. product_name (상품명)
3. reviewer_name (리뷰 등록자명)
4. review_title (리뷰 제목)
5. review_text (리뷰 내용)
6. review_rating (상품 리뷰 별점)
7. verified_purchase (실구매 확인 여부)
8. review_date (리뷰 작성 날짜)
9. helpful_count (해당 리뷰가 도움이 된 사람의 수)
10. uniq_id (리뷰 특정 ID)
11. scraped_at (해당 리뷰를 가져온 시점)

In [2]:
# 데이터 읽어와 로딩 후 출력 진행

print("파일 읽기 실행 중입니다...\n")
start = time.time() # 로딩 시작 시간 저장
df = pd.read_csv('/workspaces/ml202401-final-LeeEulHo/amazon_uk_shoes_products_dataset_2021_12.csv') # 파일 경로 지정
end = time.time() # 로딩 종료 시간 저장
print("파일 로딩에 걸린 시간은 " + str(round(end - start, 2)) + " 초입니다.") # 걸린 시간 출력

df # 데이터 프레임 원본 출력

파일 읽기 실행 중입니다...

파일 로딩에 걸린 시간은 0.06 초입니다.


,url,product_name,reviewer_name,review_title,review_text,review_rating,verified_purchase,review_date,helpful_count,uniq_id,scraped_at
0,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Jocelyn McSayles,Love em,Love these. Was looking for converses and thes...,5.0,True,Reviewed in the United States on 2 June 2020,2 people found this helpful,36eae4e5-2894-5279-a0b7-d2b330e2b814,24/12/2021 02:26:25
1,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Kenia Rivera,The plastic ripped,"The shoes are very cute, but after the 2nd day...",2.0,True,Reviewed in the United States on 28 October 2021,NaN,f4778bb8-3070-5cb1-b5aa-ffce41a97b57,24/12/2021 02:26:25
2,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Chris Souza,Good quality,Good quality,5.0,True,Reviewed in the United States on 20 January 2021,NaN,db5a7525-d40b-5265-84d8-df4f29837a3b,24/12/2021 02:26:25
3,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Amazon Customer,Good,Great,5.0,True,Reviewed in the United States on 22 April 2021,NaN,75a42851-6462-54b5-988a-27d336221943,24/12/2021 02:26:25
4,https://www.amazon.co.uk/dp/B08SW434MG,"GUESS Women's Bradly Gymnastics Shoe, White, 7 UK",Graziella,PERFETTE!!,Ho scelto il modello bianco con rifinitura die...,5.0,True,Reviewed in Italy on 2 April 2021,2 people found this helpful,232dee43-849e-5d06-ba05-efb3f4814714,24/12/2021 02:26:25
...,...,...,...,...,...,...,...,...,...,...,...
6818,https://www.amazon.co.uk/dp/B07TPYWFVN,"Clarks Vennor Wing, Men’s Low-Top Sneakers, Bl...",mauti72,Schick und leicht,"Schicker Schuh, läuft sich gut.",5.0,True,Reviewed in Germany on 15 October 2020,NaN,0850eae1-fa2f-59e6-bf30-ad49151bfa20,24/12/2021 02:29:39
6819,https://www.amazon.co.uk/dp/B07TPYWFVN,"Clarks Vennor Wing, Men’s Low-Top Sneakers, Bl...",Charles Lechesnier,EXCELLENT,Mieux que je ne l'imaginais. Très bonne taille...,5.0,True,Reviewed in France on 23 August 2020,NaN,4bf117ed-ea7c-517c-967c-2aee3f80ed29,24/12/2021 02:29:39
6820,https://www.amazon.co.uk/dp/B084WB2D93,"Rohde Men's Tivoli-H Mule, 82 Anthracite, 12.5 UK",Rebecca Lützenkirchen,Einfach schöne Hausschuhe,Habe sie als Geschenk gekauft und sie sind seh...,5.0,True,Reviewed in Germany on 4 October 2021,NaN,5b129eb2-a438-5377-9c46-217a177615b2,24/12/2021 02:29:39
6821,https://www.amazon.co.uk/dp/B084WB2D93,"Rohde Men's Tivoli-H Mule, 82 Anthracite, 12.5 UK",Sergej Friedel,Langlebig.,Trage diese Hausschuhe fast zwei Monate jeden ...,5.0,True,Reviewed in Germany on 31 January 2021,NaN,91144305-98db-5a55-8ec4-16a253beb811,24/12/2021 02:29:39


In [3]:
# 데이터의 구조 확인 및 출력

print("데이터의 구조(행, 열)은 : " + str(df.shape) + " 입니다") #df.shape를 string 형태로 변경하여 출력

데이터의 구조(행, 열)은 : (6823, 11) 입니다


### 2) 결측치 제거

In [4]:
# 결측치 여부 확인

df.isnull().sum() # 결측치 값이 존재하는 데이터 갯수 출력

url                     0
product_name            0
reviewer_name           0
review_title            1
review_text             9
review_rating           0
verified_purchase       0
review_date             0
helpful_count        4870
uniq_id                 0
scraped_at              0
dtype: int64

확인 결과 helpful_count와 review_title, review_text에 존재하기에 helpful_count는 열을 제거하고, 이후에 결측치를 가진 행을 제거하는 것으로 진행함. 

In [5]:
df = df.drop(["helpful_count"], axis=1) # helpful_count 열 제거
df = df.dropna(axis=0) # 결측치 존재하는 행 제거

df.isnull().sum() # 결측치 여부 확인

url                  0
product_name         0
reviewer_name        0
review_title         0
review_text          0
review_rating        0
verified_purchase    0
review_date          0
uniq_id              0
scraped_at           0
dtype: int64

In [6]:
# 결측치 제거 후 데이터 갯수 확인

df.count() # 데이터 갯수 출력

url                  6813
product_name         6813
reviewer_name        6813
review_title         6813
review_text          6813
review_rating        6813
verified_purchase    6813
review_date          6813
uniq_id              6813
scraped_at           6813
dtype: int64

즉, 결측치와 문제가 될 만한 helpful_count를 제거한 결과(열, 행)가 (6823, 11)에서 (6813, 10)으로 변경되어 완료된 것을 확인 가능하다.

그렇다면 다음은 우리가 모델을 통해 얻을 내용인, 신발의 상품 별 리뷰의 수를 확인하기 위해 각 제품별 리뷰의 갯수를 확인 해보도록 하겠다.

### 3) 상품별 리뷰 갯수 확인

In [7]:
# 상품별 리뷰 갯수 확인

products_freq = df["product_name"].value_counts()
print("총 " + str(products_freq.size) + "개 상품의 리뷰 존재")
print(products_freq)


총 1086개 상품의 리뷰 존재
product_name
Teva K Hurricane 3, Balboa Sodalite Blue, 12 UK Child                                  10
Think! Men's Kong_585653 Derbys, (Sz/Kombi 09), 6.5 UK                                 10
Reebok Women's Princess Sneaker, White/White/Collegiate Royal, 6 UK                    10
PUMA Men's Axelion Ultra Cross Trainer, Black, Signal red, 10.5 UK                     10
Geox boy J MUNFREY C Low-Top Sneakers, Grey (Grey/Orange C0036), 26                    10
                                                                                       ..
Aldo Women's RPPLFROST1B Sneaker, Light Pink, 6 UK                                      1
Dr. Brinkmann Women’s Flat Platform Size: 6 UK Blue                                     1
adidas Originals Unisex VRX Low Skate Shoe, white/black/white, 4 M US Big Kid           1
Aigle Unisex Adults Brea Botte Iso Wellington Boots, Blue (Marine New 001), 10.5 UK     1
Fila Mindblower Gold Fusion/White-Navy                               

In [8]:
# 리뷰의 갯수 상위 10개 출력

# 모두 동일하게 많아야 10개의 리뷰를 가졌기에 분류에 의미가 없어서, 이후에 첫 단어를 추출하여 브랜드 별로 다시 상위 10개를 추출함

print("가장 리뷰가 많은 상품은 " + str(products_freq.index[0]).strip() + "이며 " + str(products_freq[0]) + "개의 리뷰\n")
print("이후 10개의 많은 리뷰를 가진 상품들 : ")
print(products_freq[1:11].to_string())

가장 리뷰가 많은 상품은 Teva K Hurricane 3, Balboa Sodalite Blue, 12 UK Child이며 10개의 리뷰

이후 10개의 많은 리뷰를 가진 상품들 : 
product_name
Think! Men's Kong_585653 Derbys, (Sz/Kombi 09), 6.5 UK                                10
Reebok Women's Princess Sneaker, White/White/Collegiate Royal, 6 UK                   10
PUMA Men's Axelion Ultra Cross Trainer, Black, Signal red, 10.5 UK                    10
Geox boy J MUNFREY C Low-Top Sneakers, Grey (Grey/Orange C0036), 26                   10
Reebok Men's Club MEMT Wide 4e Sneaker, Black/DGH Solid Grey, 6.5 UK                  10
Propet Women's Ladybug Walking Shoe, Oyster, 11 W US                                  10
MSMAX Black Patent Character Mary Jane Flexible Dance Tap Shoes Little Kid Size 11    10
Skechers Kids Girls' Slip on Water Shoe, Champagne, 5 Big Kid M                       10
Reebok Kids Royal Cljog 2 Sneaker                                                     10
New Balance Kids&#39; 574v1 Sport Sneaker                                         

/tmp/ipykernel_97916/3718880405.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("가장 리뷰가 많은 상품은 " + str(products_freq.index[0]).strip() + "이며 " + str(products_freq[0]) + "개의 리뷰\n")


### 4) 실구매 여부 확인

In [9]:
# 실구매 여부가 False인 데이터가 있는지 여부를 확인

print(df["verified_purchase"] == "true")

0       False
1       False
2       False
3       False
4       False
        ...  
6818    False
6819    False
6820    False
6821    False
6822    False
Name: verified_purchase, Length: 6813, dtype: bool


위 결과로 실구매 작성자만 리뷰를 작성함을 확인 가능함

### 5) 동일 리뷰 작성자가 작성한 리뷰 유무 확인

In [10]:
# 동일한 작성자의 리뷰가 존재하는지 여부 확인

reviewer_freq = df["reviewer_name"].value_counts()
print("총 " + str(reviewer_freq.size) + "명이 작성한 상품의 리뷰 존재")
print(reviewer_freq)

총 5516명이 작성한 상품의 리뷰 존재
reviewer_name
Amazon Customer      318
Cliente Amazon       132
Amazon Kunde          97
Cliente de Amazon     28
Client d'Amazon       28
                    ... 
A Sobol                1
Collin Schwantes       1
Josephine Miller       1
alberto_oter           1
Swidurski              1
Name: count, Length: 5516, dtype: int64


In [11]:
# 리뷰 작성이 가장 많은 상위 11명 작성자 확인

print("가장 작성한 리뷰가 많은 작성자는 " + str(reviewer_freq.index[0]).strip() + "이며 " + str(reviewer_freq[0]) + "개의 리뷰\n")
print("이후 10개의 많은 리뷰를 가진 상품들")
print(reviewer_freq[1:11].to_string())

가장 작성한 리뷰가 많은 작성자는 Amazon Customer이며 318개의 리뷰

이후 10개의 많은 리뷰를 가진 상품들
reviewer_name
Cliente Amazon       132
Amazon Kunde          97
Cliente de Amazon     28
Client d'Amazon       28
Kindle Customer       23
Alex                  10
Lisa                   8
Stephanie              8
Daniel                 8
John                   7


/tmp/ipykernel_97916/591348586.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("가장 작성한 리뷰가 많은 작성자는 " + str(reviewer_freq.index[0]).strip() + "이며 " + str(reviewer_freq[0]) + "개의 리뷰\n")


### (1 ~ 5) 데이터 분석 결과

신발 상품에 대한 리뷰가 특정 상품에 많은 것은 비교적 크게 차이를 보이지 않고 10개가 최대임을 확인 가능하기에 분류하여 어떤 상품이 더 긍정적인 리뷰를 가지는지는 확인하기에는 큰 의미가 없다고 판단이
되며, 실구매 여부도 역시 모든 리뷰를 확인한 결과 TRUE이기 때문에 확인을 재차 할 필요성을 못느낌. 

하지만, 작성자가 동일한 리뷰 작성자가 다수의 리뷰를 작성한 것은 확인되었기 때문에 리뷰의 감정을 분석하는 부분에서 어느 정도 특정 리뷰 작성자의 리뷰 작성 성격이 어떤지에 대한 결과는 확인 가능할 것으로 보임.

바로 다음 데이터 전처리 과정으로 넘어가도록 하겠음.

### 6) 데이터 프레임 customization

기존의 데이터 프레임을 확인하면 하단과 같이 데이터가 존재한다.

1. url (상품 URL 주소)
2. product_name (상품명)
3. reviewer_name (리뷰 등록자명)
4. review_title (리뷰 제목)
5. review_text (리뷰 내용)
6. review_rating (상품 리뷰 별점)
7. verified_purchase (실구매 확인 여부)
8. review_date (리뷰 작성 날짜)
9. helpful_count (해당 리뷰가 도움이 된 사람의 수)
10. uniq_id (리뷰 특정 ID)
11. scraped_at (해당 리뷰를 가져온 시점)

그러나 필요한 것은 product_name, reviewer_name, review_title, review_text, review_rating, review_date 정도이기에 이를 제외한 url, verified_purchase, uniq_id, scraped_at는 삭제하여 데이터 프레임을 수정 작업을 진행한다.


In [12]:
# 기존의 데이터 프레임에서 필요 없는 열을 삭제 진행

df = df.drop(["url", "verified_purchase", "uniq_id", "scraped_at"], axis=1) # 불필요한 열 삭제

df.head()

,product_name,reviewer_name,review_title,review_text,review_rating,review_date
0,Klasified Women's Transparent Clear Sneaker Sh...,Jocelyn McSayles,Love em,Love these. Was looking for converses and thes...,5.0,Reviewed in the United States on 2 June 2020
1,Klasified Women's Transparent Clear Sneaker Sh...,Kenia Rivera,The plastic ripped,"The shoes are very cute, but after the 2nd day...",2.0,Reviewed in the United States on 28 October 2021
2,Klasified Women's Transparent Clear Sneaker Sh...,Chris Souza,Good quality,Good quality,5.0,Reviewed in the United States on 20 January 2021
3,Klasified Women's Transparent Clear Sneaker Sh...,Amazon Customer,Good,Great,5.0,Reviewed in the United States on 22 April 2021
4,"GUESS Women's Bradly Gymnastics Shoe, White, 7 UK",Graziella,PERFETTE!!,Ho scelto il modello bianco con rifinitura die...,5.0,Reviewed in Italy on 2 April 2021


### 7) Initial column processing (상품명 변경)

상품별 상세 명칭이 너무 많기 때문에 제조사가 동일한 상품의 리뷰가 가장 많은 상위 10개의 브랜드명으로 묶어서 상품명을 변경한다.

In [13]:
# 가장 많은 상품 리뷰를 가진 브랜드 명 추출

first_words = df["product_name"].str.split().str[0]
first_words.value_counts().head(10)

product_name
adidas      896
New         546
PUMA        500
Reebok      346
Skechers    275
Merrell     185
ASICS       168
Geox        143
Clarks      118
Converse     89
Name: count, dtype: int64

In [14]:
# 상위 10개에 해당하는 상품 브랜드로 상품명을 변경하는 함수 생성

# 만약 product_name에 데이터가 하단의 입력된 브랜드명이 존재하면 그 브랜드 명으로 변경하는 코드로서 함수로 정의함.

def replace_product_name(row):
    if "adidas" in row["product_name"]:
        return "adidas"
    elif "New Balance" in row["product_name"]:
        return "New Balance"
    elif "PUMA" in row["product_name"]:
        return "PUMA"
    elif "Reebok" in row["product_name"]:
        return "Reebok"
    elif "Skechers" in row["product_name"]:
        return "Skechers"
    elif "Merrell" in row["product_name"]:
        return "Merrell"
    elif "ASICS" in row["product_name"]:
        return "ASICS"
    elif "Geox" in row["product_name"]:
        return "Geox"
    elif "Clarks" in row["product_name"]:
        return "Clarks"
    elif "Converse" in row["product_name"]:
        return "Converse"
    else:
        return row.product_name

In [15]:
# 기존의 데이터에 위에서 지정한 이름 변경 함수를 적용

df["shoes_product_df"] = df.apply(replace_product_name, axis = 1)

In [18]:
# 위에서 지정한 상위 10개 제품 브랜드에 해당하는 리뷰 갯수 count 하고 확인

print(df["shoes_product_df"].value_counts().head(10)) # 제대로 바뀐 것을 확인 가능함

shoes_product_df
adidas         896
New Balance    546
PUMA           500
Reebok         346
Skechers       275
Merrell        185
ASICS          168
Geox           143
Clarks         118
Converse        89
Name: count, dtype: int64


## 2. 모델 구현

### 1) stop word 제거

부정/긍정 리뷰 평가에 부수적인 감정을 가지는 stop word는 필요 없으며, 분석 시간 효율 저하의 효과가 있기 때문에 제거를 진행한다.

필요 모듈 설치 및 모델 선택 및 감정 분석기에 필요한 자료 다운로드 진행

In [21]:
%pip install nltk #pip를 통해 nltk 설치
import nltk as nltk # nltk import
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer # nltk의 VADER 감정 분석기 impot 진행
nltk.download('vader_lexicon') # VADER 모델의 감정 분석기에 필요한 어휘 자료를 다운로드 진행

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [22]:
# stopword 리스트를 외부에서 가져와서 지금의 리뷰에서 존재하는 stopword를 제거하는 과정을 여기에서 진행함

nltk.download('stopwords')

start = time.time()
cache = set(stopwords.words("english"))
def remove_stopwords(review):
    text = " ".join([word for word in review.split() if word not in cache])
    return text

# Remove the stop words from both columns
df.review_title = df.review_title.apply(remove_stopwords)   
df.review_text = df.review_text.apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 3. 모델 학습 및 튜닝

In [25]:
vader_sentiment = SentimentIntensityAnalyzer() # VADER 감정 분석기를 초기화

def calc_sentiment(review):    #계산을 위한 함수를 지정
    if review == "No Negative" or review == "No Positive": # 리뷰가 "No Negative" 또는 "No Positive"인 경우
        return 0
    return vader_sentiment.polarity_scores(review)["compound"]  # 리뷰가 텍스트인 경우 VADER 감정 분석기를 사용하여 감정 점수(컴파운드 스코어)를 계산하고 반환

In [26]:
print("Calculating sentiment columns for both positive and negative reviews")
start = time.time()
df["Negative_Sentiment"] = df.Negative_Review.apply(calc_sentiment)
df["Positive_Sentiment"] = df.Positive_Review.apply(calc_sentiment)
end = time.time()
print("Calculating sentiment took " + str(round(end - start, 2)) + " seconds")

Calculating sentiment columns for both positive and negative reviews


AttributeError: 'DataFrame' object has no attribute 'Negative_Review'

## 4. 모델 평가

## 5. 시각화 및 보고